# Data Exploration 
Now that we have the streamflow data imported as a CSV file, we move on to the next step in the analytical workflow, namely exploring the data. After we read the data in CSV file back into our session, we will repeat the exploratory analyses we did in Excel:

* Create some scatter plots of the entire dataset as well as for the pre- and post-Falls lake years.


* Deriving some new column in our data frame from existing columns:
 * Create `year` and `month` columns from our `datetime` data
 * Create a `water_year` column based on year and month values
 * Convert our *cfs* data into *cms* and *mgd* units and re-plot


* Summarize the data by *confidence codes*:
 * Group the data by data-value qualification (i.e., "confidence") codes
 * Determine how many different codes exist in our data and what those codes are
 * Compute the number of records in each code
 * Create a quick bar pot showing the record count in each category


* Compute summary statistics of the entire set of discharge data.
 * Compute min, max, mean, median, as well as specific quantiles.
 * Plot these values
 * Repeat for select years of data (pre- and post- Falls Lake construction)

## Import libraries
First, we need to load a few Python packages build for data analyisis projects.

In [1]:
#Import modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Instruct Jupyter to allow plots in the document itself
%matplotlib inline

## Read the data into a *Pandas* dataframe.
Read in the CSV created in the previous notebook into a new Pandas dataframe named `df`. We'll also force the data type of the `site_no` field to be a *string*, not a *number*, which Pandas would infer by default...

In [3]:
#Load the saved csv file, reading the site_no column as a string, not a number.
df = pd.read_csv('GageData.csv', dtype={'site_no':'str'}) 

In [4]:
#Confirm it looks good by viewing the first 5 records using the "head()" function
df.head()

,agency_cd,site_no,datetime,MeanFlow_cfs,Confidence
0,USGS,02087500,1930-10-01,347.0,A
1,USGS,02087500,1930-10-02,173.0,A
2,USGS,02087500,1930-10-03,132.0,A
3,USGS,02087500,1930-10-04,125.0,A
4,USGS,02087500,1930-10-05,125.0,A


### Data types
Dataframes are structured so that each column contains values of a constant and defined **data type**. Typical data types in a Pandas dataframe include integers (`int64`), floating point numbers (`float64`), strings (`object`), and date/time objects (`datetime64`).

If we import data into a dataframe, it infers data types from the the values in the input file unless we override the defaults, as we did above with the `site_no` varible. We can view the datatypes in a dataframe via the dataframe's `dtypes` property:

In [5]:
#Show the data types of each column
df.dtypes

agency_cd        object
site_no          object
datetime         object
MeanFlow_cfs    float64
Confidence       object
dtype: object

Note that the `datetime` variable has an incorrectly assigned data type. It is listed as a string (`object`), so we need to re-define it to be a `datetime64` object, specifying the date format our data use.

In [6]:
#Convert datetime to an actual datetime object
df['datetime'] = pd.to_datetime(df['datetime'],format=('%Y-%m-%d'))
df.dtypes

agency_cd               object
site_no                 object
datetime        datetime64[ns]
MeanFlow_cfs           float64
Confidence              object
dtype: object

### Indexes and data slices
Dataframe *indexes* provide different ways to select, sort, and organize data rows. By setting our `datetime` field as an index, we can then select rows by date ranges or "slices". 

In [7]:
#Setting the date time as the index allows time slicing
df.index = df.datetime

In [8]:
#Create two new dataframe views: One with records before Falls Lake an one after
dfPre = df['1930-01-01':'1979-12-31']
dfPost = df['1984-01-01':'2017-12-31']

## Create time series plots of daily discharge data
With our data properly imported and formatted, we can now create some exploratory plots of our data, which may reveal any gaps or outliers of concern. 

Pandas has some plotting capability that will examine. However, we'll also explore the *matplotlib* package, a powerful plotting package that has both Python and R versions, as well as *seaborn* which adds nice aesthetics to our plots. 

#### Create a simple plot using `Pandas` functionality. 
See https://pandas.pydata.org/pandas-docs/stable/visualization.html

In [9]:
#Plot using Pandas
df.plot();

ValueError: ordinal must be >= 1

ValueError: ordinal must be >= 1

#### Create a time series plot of mean discharge using `matplotlib`
https://matplotlib.org/devdocs/api/pyplot_summary.html

In [ ]:
#Create a plot canvas of a specified size
plt.figure(figsize=(20,6));

#Plot the mean flow (against time)
plt.plot(df['MeanFlow_cfs']);

#Add a vertical line indicating when Falls Lake construction began
plt.axvline(x='1980-01-01',color='red',ls='--');

#Add some aesthetics
plt.title("Neuse River Near Goldsboro, NC");
plt.ylabel("Discharge (cfs)");

#### Invoke `Seaborn` to make prettier plots 
https://seaborn.pydata.org/tutorial/aesthetics.html


In [ ]:
#Activate seaborn default aesthetics
sns.set(font_scale=2)
#Repeat above
plt.figure(figsize=(20,6));
plt.plot(df['MeanFlow_cfs']);
plt.axvline(x='1980-01-01',color='red',ls='--');
plt.title("Neuse River Near Goldsboro, NC");
plt.ylabel("Discharge (cfs)");

### Create a plot of all data, superimposed with the pre- and post-Falls Lake data views


In [ ]:
#Create the canvas
plt.figure(figsize=(20,6))

#Add plot lines for the entire dataframe, then the two dataframe subsets
plt.plot(df['MeanFlow_cfs'],color='grey',linewidth=0.1) 
plt.plot(dfPre['MeanFlow_cfs'],color='green',alpha=0.5,linewidth=0.5)
plt.plot(dfPost['MeanFlow_cfs'],color='blue',alpha=0.5,linewidth=0.5)

#Add the aesthetics
plt.axvline(x='1979-12-31',color='red',ls='--')
plt.title("Neuse River Near Goldsboro, NC")
plt.ylabel("Discharge (cfs)");
plt.annotate('Construction begins',
             xy=('1980',16000),
             xytext=('1955',18000),
             arrowprops=dict(facecolor='red',shrink=0.05))

#Show the plot
plt.show;

## Creating derived columns
Here we: (1) derive new *year* and *month* columns from our datetime data; (2) compute a *water year* column; (2) compute mean flow in cubic meters per second (cms); and (3) replot.

### Deriving `year` and `month` columns from the `datetime` data
We'll be working with data by year and month, so we need to add those as new columns to our dataframe. The year and month of each record is held in the `datetime` field, so we just need to extract those data into new columns. Pandas' `DatetimeIndex` function is designed to do just that:

In [ ]:
#Extract years and months into a new columns in our dataframe
df['year'] = pd.DatetimeIndex(df['datetime']).year
df['month'] = pd.DatetimeIndex(df['datetime']).month
#Have a look...
df.head()

### Adding a `water_year` column. 
Water year begins in October. Therefore it equals the calendar year for months prior to October, but for October, November, and December, it takes the value of the following year. 

There are two methods for computing water year. Both yield the exact same results but show different data techniques using Pandas. 

##### ♦ Method 1 for computing water year: computing values on filtered records
In the first method, we add a new column `water_year` to our data frame, setting its default value to the value in the `year` column. Then we filter for records where the month is >= 10 and revise the value in `water_year` column to be `year + 1`.

In [ ]:
#Create a new column, setting all its values the same as the year column
df['water_year'] = df['year']

In [ ]:
#Create a mask of just records with months from Jan thru Sept (i.e. < 10)
monthMask = df['month'] >= 10

In [ ]:
#Apply this mask to create a view of just records from Jan thru Sept
df.loc[monthMask, 'water_year'] = df['year'] + 1

In [ ]:
#Have a look at a few random records
df.sample(5)

##### ♦ Method 2 for computing water year: using a `lambda` function
In this approach, we loop through all records and apply a "lambda" function on each. In this lambda function, the value of "x" is taken from the `datetime` column and the output of the function is passed into the `water_year2` column.

In [ ]:
#Use a lambda function to assign values to water_year2 based on datetime values
df['water_year2'] = df['datetime'].apply(lambda x: x.year if x.month < 10 else x.year + 1)

In [ ]:
#Show the results; water_year and water_year2 are the same...
df.sample(10)

In [ ]:
#Drop the water_year2 column, now that the demonstration is over...
df.drop("water_year2",axis='columns',inplace=True)

### Converting cubic feet/second to cubic *meter*/second

In [ ]:
#Convert from cfs to cms (1 CFS = 0.028316847 CMS)
df['MeanFlow_mps'] = df['MeanFlow_cfs'] * 0.028316847

In [ ]:
#Replot (this time in a thinner, orange line)
plt.figure(figsize=(20,6));
plt.plot(df['datetime'],df['MeanFlow_mps'],linewidth=0.5,color='orange')
plt.ylim((0,700))
plt.show;

### ♦ Exercise ♦ : 
Convert flow in cfs to flow in million gallons per day (MGD) and replot.

In [ ]:
#Convert from csf to mgd (1 CFS = 0.53817 MGD)

In [ ]:
#Replot: see if you can make the line green

## Summarize our data

In [ ]:
#Get a count of all records from the dataframe's shape (rows, columns)
df.shape

In [ ]:
#Or just show the rows, i.e., the first item in the shape result
df.shape[0]

In [ ]:
#We can also use the Python `len` command to show the "lenght" of the dataframe
len(df)

### Summarizing/grouping records by Confidence code
Here, we group the data by the unique values in a column, namely the `Confidence` column. First, we'll just examine the number of unique values and what those values are. 

In [ ]:
#Use nunique on the column to list the number of unique values
print(df['Confidence'].nunique())

In [ ]:
#Use unique to show what the 4 unique values are
print(df['Confidence'].unique())

Now, we'll **group** the records by confidence codes

In [ ]:
#Create the Pandas GroupBy object
grpConfidence = df.groupby(['Confidence'])

In [ ]:
#We can now list the counts of records by confidence code
grpConfidence.count()

In [ ]:
#Or we can just show the count by a single column
grpConfidence['MeanFlow_cfs'].count()

### Plot counts by confidence code
See: https://pandas.pydata.org/pandas-docs/stable/visualization.html

In [ ]:
#Compute counts and plot them as a bar chart
count_by_Confidence = grpConfidence['MeanFlow_cfs'].count()
count_by_Confidence.plot(kind='pie');

### Generating summary statistics with `Describe`

In [ ]:
#Summarize the MeanFlow_cfs data, using default outputs
df['MeanFlow_cfs'].describe()

In [ ]:
#Summarize, using our own percentiles
sumAll = df['MeanFlow_cfs'].describe(percentiles=[0.1,0.25,0.75,0.9])
sumAll

In [ ]:
#Describe records before 1980 and after 1984 (using index slicing, as above)
sumPre = dfPre['MeanFlow_cfs'].describe(percentiles=[0.1,0.25,0.75,0.9])
sumPost = dfPost['MeanFlow_cfs'].describe(percentiles=[0.1,0.25,0.75,0.9])

In [ ]:
#Concatenate (or join) the pre and post summary objects
dfSummary = pd.concat([sumAll,sumPre,sumPost],axis=1)
dfSummary.columns = ("all","before","after")

In [ ]:
#Plot bar charts of the three summary tables
dfSummary[4:9].plot(kind='bar',
                    figsize=(20,6),
                    title="Data Summaries: Quantiles");

## Monthly plots
To produce monthly plots, we need to group our data by month, then plot. Grouping by month requires us to extract the month from the datetime column. 

In [ ]:
#Extract the month from the datetime column
df['Mo'] = df['datetime'].map(lambda x: x.month)
df.head()

In [ ]:
byMonth = df.groupby('Mo')

In [ ]:
monthlyDF = byMonth['MeanFlow_cfs'].mean()

In [ ]:
monthlyDF

In [ ]:
monthlyDF.plot(figsize=(20,6),legend=True);

## ♦Exercise♦: Annual plots
* Add a year column to your dataframe
* Group by year
* Compute the total (sum) annual discharge
* Create a plot

In [ ]:
#Add a year column


In [ ]:
#Group by year


In [ ]:
#Compute total annual discharge


In [ ]:
#Plot
